# Replicate previous paper

[This github repository](https://github.com/yashsmehta/personality-prediction#predicting-personality-on-unseen-text) contains code for the paper [Bottom-Up and Top-Down: Predicting Personality with Psycholinguistic and Language Model Features](https://ieeexplore.ieee.org/document/9338428), where the authors propose a novel deep learning-based model which integrates traditional psycholinguistic features with language model embeddings to predict personality from the Essays dataset for Big-Five and Kaggle dataset for MBTI.

We used this paper's result as a baseline of model performance for personality detection.

In [1]:
"""
Try this code only at first time
"""
# Clone the git hub repo onto my Google Drive working folder
#!git clone 'https://github.com/Yuta555/personality-prediction'

Cloning into 'personality-prediction'...
remote: Enumerating objects: 962, done.
remote: Counting objects: 100% (962/962), done.
remote: Compressing objects: 100% (364/364), done.
remote: Total 962 (delta 582), reused 945 (delta 580), pack-reused 0
Receiving objects: 100% (962/962), 53.49 MiB | 6.10 MiB/s, done.
Resolving deltas: 100% (582/582), done.
Updating files: 100% (58/58), done.


In [2]:
!pip install -q -U transformers tweet-preprocessor sentencepiece python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 15.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import warnings

In [ ]:
%cd personality-prediction

## Split data into train/test dataset

In [4]:
SEED = 42

df = pd.read_csv('data/kaggle/kaggle.csv')

train_df, test_df = train_test_split(df, test_size=0.1, random_state=SEED)
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_df.to_csv('data/kaggle/kaggle_train.csv', index=False)
test_df.to_csv('data/kaggle/kaggle_test.csv', index=False)

## Extract features from text data using BERT model

In [6]:
!python LM_extractor.py -dataset_type 'kaggle' -token_length 512 -batch_size 32 -embed 'bert-base' -op_dir 'pkl_data' -kaggle_train True

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `sr

## Fine-tune detection model (MLP)

In [7]:
!python finetune_models/MLP_LM.py -dataset "kaggle" -save_model "yes"

2023-11-12 23:09:06.877705: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-12 23:09:06.877753: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-12 23:09:06.877795: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
kaggle : bert-base : 11 : 512_head : cls
/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
{'acc': [76.824581

## Predict personality on test dataset

In [ ]:
%cd personality-prediction

In [13]:
from .unseen_predictor import predict

embed = "bert-base"
op_dir = "pkl_data/"
token_length = 512
finetune_model = "mlp_lm"
dataset = "kaggle"

tqdm.pandas()

def labeling(pred_dict):
    label = ""
    label += "E" if pred_dict['E'] >= 0.5 else "I"
    label += "N" if pred_dict['N'] >= 0.5 else "S"
    label += "F" if pred_dict['F'] >= 0.5 else "T"
    label += "J" if pred_dict['J'] >= 0.5 else "P"
    return label

df_test = pd.read_csv('data/kaggle/kaggle_test.csv')

preds = df_test['text'].progress_apply(lambda x: predict(x, embed, op_dir, token_length, finetune_model, dataset))
pred_labels = preds.apply(lambda x: labeling(x))
pred_labels.name = "pred_label"

pred_labels.to_csv('../outputs/explogs/test_prediction.csv', index=False)

100%|██████████| 868/868 [36:02<00:00,  2.49s/it]


## Evaluate the result

In [1]:
%cd personality-prediction

/Users/yuta/Documents/GitHub/LLM-personality-evaluation.nosync/personality_detection_model/previous_paper/personality-prediction


In [39]:
from collections import defaultdict
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score


ref_labels = pd.read_csv('data/kaggle/kaggle_test.csv')['type']
pred_labels = pd.read_csv('explogs/test_prediction.csv')['pred_label']

# 16 classes
acc = round(accuracy_score(ref_labels, pred_labels), 3)
f1 = round(f1_score(ref_labels, pred_labels, average="macro"), 3)
pre = round(precision_score(ref_labels, pred_labels, average="macro"), 3)
rec = round(recall_score(ref_labels, pred_labels, average="macro"), 3)

print("16-class model")
print(f"- Accuracy: {acc}")
print(f"- F1 score: {f1}")
print(f"- Precision: {pre}")
print(f"- Recall: {rec}")

# Four binary classes
metrics_dict = {"Accuracy": [], "F1 score": [], "Precision": [], "Recall": []}

ref_labels_sep = pd.DataFrame([list(label) for label in ref_labels])
pred_labels_sep = pd.DataFrame([list(label) for label in pred_labels])

columns = ["1st_dim", "2nd_dim", "3rd_dim", "4th_dim"]
ref_labels_sep.columns = columns
pred_labels_sep.columns = columns

for col in columns:
    ref = [1 if label in ["I", "F", "N", "J"] else 0 for label in ref_labels_sep[col]]
    pred = [1 if label in ["I", "F", "N", "J"] else 0 for label in pred_labels_sep[col]]
    
    metrics_dict["Accuracy"].append(accuracy_score(ref, pred))
    metrics_dict["F1 score"].append(f1_score(ref, pred))
    metrics_dict["Precision"].append(precision_score(ref, pred))
    metrics_dict["Recall"].append(recall_score(ref, pred))

avg_acc = round(sum(metrics_dict["Accuracy"])/4, 3)
avg_f1 = round(sum(metrics_dict["F1 score"])/4, 3)
avg_pre = round(sum(metrics_dict["Precision"])/4, 3)
avg_rec = round(sum(metrics_dict["Recall"])/4, 3)

print()
print("Binary class model - Average score across 4 dimensions")
print(f"- Accuracy: {avg_acc}")
print(f"- F1 score: {avg_f1}")
print(f"- Precision: {avg_pre}")
print(f"- Recall: {avg_rec}")

16-class model
- Accuracy: 0.294
- F1 score: 0.087
- Precision: 0.141
- Recall: 0.12

Binary class model - Average score across 4 dimensions
- Accuracy: 0.731
- F1 score: 0.632
- Precision: 0.78
- Recall: 0.655


/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
